In [1]:
import re
import pandas as pd
from nltk.corpus import stopwords

import copy
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import SMOTE
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
#from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

Using TensorFlow backend.


In [2]:
news = pd.read_excel("C:\\Users\\Kratika\\Downloads\\News333.xlsx")
news.head()

,Title,Body,Category,MegaCategory
0,BS-VI transition may lead to dumping of old st...,BS-VI transition may lead to dumping of old st...,Annual Report - Comments made in the Annual Re...,Financial
1,Annual Report 2016-2017 of Bajaj Finserv Limited,It is a broadly described annual report of Baj...,Annual Report - Comments made in the Annual Re...,Financial
2,Annual Report 2017-2018 of Eicher Motors Limit...,It is a broadly described annual report of\nEi...,Annual Report - Comments made in the Annual Re...,Financial
3,Hindalco Industries : Chairman Kumar Mangalam ...,(You can enter multiple email addresses separa...,Annual Report - Comments made in the Annual Re...,Financial
4,"WIPRO continues it's uptrend, although on a we...","WIPRO continues it's uptrend, although on a we...",Annual Report - Comments made in the Annual Re...,Financial


In [3]:
from nltk.tokenize import word_tokenize 

def remove_stop_words(data):
    stop_words = set(stopwords.words('english'))
    stop_words.union(['january','february','march','april','may','june','july','august','september','october','november','december'])
    stop_words.union(['jan','feb','mar','apr','may','jun','jul','aug','sept','oct','nov','dec'])
    stop_words.union(['monday', 'tuesday', 'wednesday', 'thursday','friday','saturday','sunday'])
    stop_words.union(['am', 'pm'])
    word_tokens = word_tokenize(data) 
    filtered_sentence = []
    
    for w in word_tokens: 
        if w.lower() not in stop_words: 
            filtered_sentence.append(w) 
    
    return " ".join(filtered_sentence)

In [4]:
news['text'] = news.Title + " " + news.Body

In [5]:

news['clean_title'] = news['Title'].apply(remove_stop_words)
news['clean_body'] = news['Body'].apply(remove_stop_words)

news['clean_text'] = news['text'].apply(remove_stop_words) 

#remove_stop_words_Body


In [6]:
# ## Split data
print("\nSplitting data")

title_tr, title_te, Category_tr, Category_te = train_test_split(news['clean_text'], news.Category,test_size =.1)
#title_tr, title_de, MegaCategory_tr , MegaCategory_de = train_test_split(title_tr,MegaCategory_tr,test_size =.07)

title_de = title_te
Category_de = Category_te

print("Training: ",len(title_tr))
print("Developement: ",len(title_de),)
print("Testing: ",len(title_te))


Splitting data
Training:  2230
Developement:  248
Testing:  248


In [7]:
from sklearn.preprocessing import LabelEncoder

# # Data Preprocessing
# ## Vectorization of data
# Vectorize the data using Bag of words (BOW)
print("\nVectorizing data")
tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+")
stop_words = nltk.corpus.stopwords.words("english")
vectorizer = CountVectorizer(tokenizer=tokenizer.tokenize, stop_words=stop_words)

vectorizer.fit(iter(title_tr))
Xtr = vectorizer.transform(iter(title_tr))
Xde = vectorizer.transform(iter(title_de))
Xte = vectorizer.transform(iter(title_te))

encoder = LabelEncoder()
encoder.fit(Category_tr)
Ytr = encoder.transform(Category_tr)
Yde = encoder.transform(Category_de)
Yte = encoder.transform(Category_te)


Vectorizing data


In [8]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [9]:
print(Ytr)

[23 42 30 ... 50 23 23]


In [10]:
Xtr

<2230x16702 sparse matrix of type '<class 'numpy.int64'>'
	with 116306 stored elements in Compressed Sparse Row format>

In [11]:
# # Train Models
# ### Baseline Model
# “stratified”: generates predictions by respecting the training set’s class distribution.
print("\n\nTraining baseline classifier")
dc = DummyClassifier(strategy="stratified")
dc.fit(Xtr, Ytr)
dc_pred = dc.predict(Xde)



Training baseline classifier


In [12]:
print(classification_report(Yde, dc_pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(dc_pred == Yde)))

                                                                                                                                    precision    recall  f1-score   support

                                                                                Annual Report - Comments made in the Annual Report       0.30      0.27      0.29        78
                                                                               Approval - Alert me when a company gets an approval       0.10      0.13      0.11        15
                                                            Award Received- Alert me when any of these companies receives an award       0.00      0.00      0.00         4
                                                             Black Swan Events - Alert me when a company faces an exceptional loss       0.06      0.08      0.07        25
                                                     Buyback of shares- Alert me when any of these companies buys back it's shares       0.

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2230, does not match size of target_names, 63
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [13]:

# ### Decision Tree
print("Training Decision tree")
dt = DecisionTreeClassifier()
dt.fit(Xtr, Ytr)
dt_pred = dt.predict(Xde)
print(classification_report(Yde, dt_pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(dt_pred == Yde)))

Training Decision tree
                                                                                                                                    precision    recall  f1-score   support

                                                                                Annual Report - Comments made in the Annual Report       0.63      0.82      0.71        78
                                                                               Approval - Alert me when a company gets an approval       0.31      0.33      0.32        15
                                                            Award Received- Alert me when any of these companies receives an award       0.00      0.00      0.00         4
                                                             Black Swan Events - Alert me when a company faces an exceptional loss       0.63      0.68      0.65        25
                                                     Buyback of shares- Alert me when any of these companies buys ba

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2230, does not match size of target_names, 63
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [14]:
# ### Random Forest
print("Training Random Forest")
rf = RandomForestClassifier(n_estimators=40)
rf.fit(Xtr, Ytr)
pred = rf.predict(Xde)
print(classification_report(Yde, pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(pred == Yde)))

Training Random Forest
                                                                                                                                    precision    recall  f1-score   support

                                                                                Annual Report - Comments made in the Annual Report       0.49      0.94      0.65        78
                                                                               Approval - Alert me when a company gets an approval       0.00      0.00      0.00        15
                                                            Award Received- Alert me when any of these companies receives an award       0.00      0.00      0.00         4
                                                             Black Swan Events - Alert me when a company faces an exceptional loss       0.42      0.88      0.57        25
                                                     Buyback of shares- Alert me when any of these companies buys ba

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2230, does not match size of target_names, 63
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [15]:

# ### Multinomial Naive Bayesian
print("Training Multinomial Naive Bayesian")
nb = MultinomialNB()
nb.fit(Xtr, Ytr)
pred_nb = nb.predict(Xde)
print(classification_report(Yde, pred_nb, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(pred_nb == Yde)))

Training Multinomial Naive Bayesian
                                                                                                                                    precision    recall  f1-score   support

                                                                                Annual Report - Comments made in the Annual Report       0.61      0.88      0.72        78
                                                                               Approval - Alert me when a company gets an approval       0.38      0.33      0.36        15
                                                            Award Received- Alert me when any of these companies receives an award       0.00      0.00      0.00         4
                                                             Black Swan Events - Alert me when a company faces an exceptional loss       0.33      0.96      0.49        25
                                                     Buyback of shares- Alert me when any of these comp

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2230, does not match size of target_names, 63
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [16]:
# ### Support Vector Classification
print("Training Support Vector Classification")
from sklearn.svm import SVC
svc = SVC()
svc.fit(Xtr, Ytr)
svc_pred = svc.predict(Xde)
print(classification_report(Yde, svc_pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(svc_pred == Yde)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Training Support Vector Classification
                                                                                                                                    precision    recall  f1-score   support

                                                                                Annual Report - Comments made in the Annual Report       0.32      1.00      0.49        78
                                                                               Approval - Alert me when a company gets an approval       0.00      0.00      0.00        15
                                                            Award Received- Alert me when any of these companies receives an award       0.00      0.00      0.00         4
                                                             Black Swan Events - Alert me when a company faces an exceptional loss       1.00      0.28      0.44        25
                                                     Buyback of shares- Alert me when any of these c

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2230, does not match size of target_names, 63
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [17]:
# ### Multilayered Perceptron
print("Training Multilayered Perceptron")
mlp = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(100, 20), random_state=1, max_iter=400)
mlp.fit(Xtr, Ytr)
mlp_pred = mlp.predict(Xde)
print(classification_report(Yde, mlp_pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(mlp_pred == Yde)))

Training Multilayered Perceptron
                                                                                                                                    precision    recall  f1-score   support

                                                                                Annual Report - Comments made in the Annual Report       0.71      0.92      0.80        78
                                                                               Approval - Alert me when a company gets an approval       0.53      0.60      0.56        15
                                                            Award Received- Alert me when any of these companies receives an award       0.00      0.00      0.00         4
                                                             Black Swan Events - Alert me when a company faces an exceptional loss       0.68      0.92      0.78        25
                                                     Buyback of shares- Alert me when any of these compani

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2230, does not match size of target_names, 63
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [18]:
print('Accuracy achieved is ' + str(np.mean(dc_pred   == Yde)))
print('Accuracy achieved is ' + str(np.mean(dt_pred   == Yde)))
print('Accuracy achieved is ' + str(np.mean(pred_nb   == Yde)))
print('Accuracy achieved is ' + str(np.mean(svc_pred  == Yde)))
print('Accuracy achieved is ' + str(np.mean(mlp_pred  == Yde)))

Accuracy achieved is 0.125
Accuracy achieved is 0.49193548387096775
Accuracy achieved is 0.5282258064516129
Accuracy achieved is 0.34274193548387094
Accuracy achieved is 0.6290322580645161


In [19]:
# # Final Model: Multilayered Perceptron
# ## Predict test data
print("\n\nPredicting test data using Multilayered Perceptron")
pred_final = mlp.predict(Xte)
print(classification_report(Yte, pred_final, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(accuracy_score(Yte,pred_final)))



Predicting test data using Multilayered Perceptron
                                                                                                                                    precision    recall  f1-score   support

                                                                                Annual Report - Comments made in the Annual Report       0.71      0.92      0.80        78
                                                                               Approval - Alert me when a company gets an approval       0.53      0.60      0.56        15
                                                            Award Received- Alert me when any of these companies receives an award       0.00      0.00      0.00         4
                                                             Black Swan Events - Alert me when a company faces an exceptional loss       0.68      0.92      0.78        25
                                                     Buyback of shares- Alert me when 

In [20]:
output = {"title":vectorizer.inverse_transform(Xte), "predicted":encoder.inverse_transform(pred_final),"actual": encoder.inverse_transform(Yte)}

In [21]:
#df = pd.DataFrame(output, columns=["title","predicted","actual"])
#df.to_csv("C:\\Users\\Kratika\\Downloads\\News333_title_predication_1.csv")

In [22]:
pred_final = mlp.predict(Xte)
print('Accuracy achieved is ' + str(np.mean(pred_final == Yte)))
output = {"text":vectorizer.inverse_transform(Xte), "predicted":encoder.inverse_transform(pred_final),"actual": encoder.inverse_transform(Yte)}
df = pd.DataFrame(output, columns=["text","predicted","actual"])


Accuracy achieved is 0.6290322580645161


In [23]:
#df = pd.DataFrame(output, columns=["text","predicted","actual"])
#df.to_csv("C:\\Users\\Kratika\\Downloads\\News333_title_body_predication.csv")

In [24]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer


#GET VECTOR COUNT
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(news.clean_text)

#SAVE WORD VECTOR
pickle.dump(count_vect.vocabulary_, open("count_vector.pkl","wb"))

In [25]:
X_train_counts

<2478x17611 sparse matrix of type '<class 'numpy.int64'>'
	with 125978 stored elements in Compressed Sparse Row format>

In [26]:
from sklearn.feature_extraction.text import TfidfTransformer

#TRANSFORM WORD VECTOR TO TF IDF
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#SAVE TF-IDF
pickle.dump(tfidf_transformer, open("tfidf.pkl","wb"))


In [27]:
X_train_tfidf

<2478x17611 sparse matrix of type '<class 'numpy.float64'>'
	with 125978 stored elements in Compressed Sparse Row format>

In [28]:
from sklearn.neural_network import MLPClassifier

#mlp =       MLPClassifier(solver='adam',  alpha=1e-5, hidden_layer_sizes=(100, 20), random_state=1, max_iter=400)
#clf_neural = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1)
clf_neural = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1)
#clf_neural = MLPClassifier(hidden_layer_sizes=(100, ), activation=’relu’, solver=’adam’, alpha=0.0001, batch_size=’auto’, learning_rate=’constant’, learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10)
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, news.MegaCategory, test_size=0.25, random_state=42)

clf_neural.fit(X_train, y_train)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(15,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='sgd', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [29]:
predicted = clf_neural.predict(X_test)
print('Accuracy achieved is ' + str(np.mean(predicted == y_test)))


Accuracy achieved is 0.4


In [30]:
print(classification_report(y_test, predicted))

                      precision    recall  f1-score   support

Awards & Recognition       0.00      0.00      0.00        10
               Event       0.00      0.00      0.00         2
              Events       0.00      0.00      0.00         1
           Financial       0.00      0.00      0.00        56
              Growth       0.00      0.00      0.00        70
       Industry News       0.00      0.00      0.00        17
         Market Buzz       0.00      0.00      0.00        28
       Negative News       0.00      0.00      0.00        51
              People       0.00      0.00      0.00        40
   Stock Performance       0.00      0.00      0.00        97
  Uncategorised News       0.40      1.00      0.57       248

            accuracy                           0.40       620
           macro avg       0.04      0.09      0.05       620
        weighted avg       0.16      0.40      0.23       620



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
mlp = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(100,), activation='relu', random_state=1, max_iter=400)

mlp.fit(X_train, y_train)
mlp_pred = mlp.predict(X_test)  
print(classification_report(y_test, mlp_pred))
print('Accuracy achieved is ' + str(np.mean(mlp_pred == y_test)))